## 참조자료 

https://stevedonovan.github.io/rustifications/2018/08/18/rust-closures-are-hard.html

# 1. 클로저

## 클로저 정의 및 호출하기 

- 주의할 점은 구조체 등은 소유권이 이동하므로 클로저 내부 사용하면 예외 발생 

In [5]:
fn main() {
    let m = 1.0;
    let c = 2.0;
    let _z = String::from("가가가"); 

    let line = |x:f64|->f64 {      // 변수에 클로저 할당할 때는 main 함수 내부에 정의 할 것
        let s = m*x + c;           // 함수의 지역변수를 클로저에서 사용이 가능 : 정수는 이동되지 않아서 사용가능 
        //let zz = _z;             // _z는 한번 소유권 이동이 되어서 계속 호출시 문제됨 ...
        s                          // 내부에서 소유권 변동을 주의해야 함  
    };
    println!("{} {}", line(0.0), line(1.0));
    println!(" m {:.2} ", m);
    // println!(" z { } ", z);
}

In [6]:
main()

2 3
 m 1.00 


()

## 1-2 클로저 축약형으로 작성하기 

### 함수와 클러저 정의 알아보기

In [31]:
fn main() {
    fn  add_one_v1   (x: u32) -> u32 { x + 1 }   // 함수 정의 
    let add_one_v2 = |x: u32| -> u32 { x + 1 };  // 클로저 정의 
    let add_one_v3 = |x|             { x + 1 };  // 클로저 정의 : 타입 에노테이션 생략 
    let add_one_v4 = |x|               x + 1  ;  // 하나의 문장이라 표현식으로 지정 
    println!(" add_one_v1 {} ", add_one_v1(100));
    println!(" add_one_v2 {} ", add_one_v2(100));
    println!(" add_one_v3 {} ", add_one_v3(100));
    println!(" add_one_v4 {} ", add_one_v4(100));
}

In [32]:
main()

 add_one_v1 101 
 add_one_v2 101 
 add_one_v3 101 
 add_one_v4 101 


()

### 클로저의 매개변수에 자료형 지정하기

In [35]:
fn main() {
    let c  = |x:u32| {x+x};

    println!(" {} ",c(10))
}

In [36]:
main()

 20 


()

## 1-3 구조체 내부에 클로저 정의하기

### 두 개의 클로저를 정의하고 하나를 다른 클로저에서 처리하기 

In [2]:
fn main() {
    let c  = |x:u32| {x+x};
    let d  = |y| c(y);

    println!(" {} ",d(10))
}

In [4]:
main();

 20 


### 클로저를 반환할 때는 캡처된 변수를 이동시키기

In [7]:
fn main() {
    
    let d  = |y| {move |x| x+y};

    println!(" {} ",d(10)(20))
}

In [8]:
main();

 30 


## 1-4 스마트 포인터로 클로저 반환하기 

In [13]:
let f: Box<dyn Fn()> = Box::new(|| {println!("foo")});

In [14]:
f()

foo


()

In [15]:
let fc: Box<dyn Fn(u32)> = Box::new(|x| {println!("foo {}",x)});

In [16]:
fc(100)

foo 100


()

In [17]:
let fc: Box<dyn Fn(u32,u32)> = Box::new(|x,y| {println!("foo {}",x+y)});

In [18]:
fc(100,200)

foo 300


()

# 2. 고차함수 처리 

## 2-1 클로저를 반환값으로 처리 

- 클로저를 반환할 때의 반환자료형은 트레이트 Fn으로 처리한다. 

### 클로저는  Fn 트레이트로 반환을 받는다

In [9]:
// This works of course
fn closure() -> impl Fn(i32) -> i32 {
     |b| b                       
}

In [10]:
closure()(100)

100

In [22]:
fn closure_1() -> impl Fn() -> () {
     || println!(" closure 1 ")
}

In [23]:
closure_1()()

 closure 1 


()

### 클로저를 반환할 때는 캡처된 자유변수는 move로 이동처리 

In [24]:
fn make_adder_1(a: i32, b: i32) -> impl Fn() -> i32 {
    move || a + b
}

In [25]:
make_adder_1(100,200)()

300

In [57]:
fn counter() -> impl FnMut() -> i32 {
    let mut value = 0;

    move || -> i32 {
        value += 1;
        return value;
    }
}

fn main() {
    let mut incre = counter();
    println!("Count 1: {}", incre());
    println!("Count 2: {}", incre());
}

In [58]:
main()

Count 1: 1
Count 2: 2


()

## 2-2  변수에 할당하기 

- 클로저를 변수에 할당할 때는 스마트 포인트로 처리
- 트레이트 객체로 지정해서 처리해야 함 


In [18]:
fn closure1() -> impl Fn(i32) -> i32 {
     |b| b                       
}

In [20]:
fn main() {
    let cl : Box<dyn Fn(i32) -> i32>  = Box::new(closure1());
    println!(" {} ", cl(100));
}

In [21]:
main()

 100 


()

## 2-3 클로저를 반환해서 처리하기

In [13]:
fn make_adder(a: i32) -> impl Fn(i32) -> i32 {
    move |b| a + b
}

fn main() {
    println!("{}", make_adder(1)(2));
}

In [14]:
main()

3


()

In [27]:
fn make_adder(a: i32) -> Box<dyn Fn(i32) -> i32> {
    Box::new(move |b| a + b)
}

fn main() {
    println!("{}", make_adder(1)(2));
}

In [28]:
main()

3


()

## 2-4 함수의 매개변수로 클로저 전달하기

In [26]:
fn func_arg (f1: impl Fn(f64)->f64,x:f64) ->  f64 {
    f1(x)
}

In [27]:
func_arg(|x| x , 100.2)

100.2

## 2-5 함수 결합하기 

In [22]:
fn compose (f1: impl Fn(f64)->f64, f2: impl Fn(f64)->f64) -> impl Fn(f64)->f64 {
    move |x| f1(f2(x))
}

fn main() {
   let f = compose(f64::sin, |x| x*x); 
   println!(" {} ", f(100.0));
}


In [23]:
main()

 -0.30561438888825215 


()

In [30]:
fn compose <T>(f1: impl Fn(T)->T, f2: impl Fn(T)->T) -> impl Fn(T)->T {
    move |x| f1(f2(x))
}

fn main() {
    let g = compose(str::trim, |s| &s[0..2]);
    println!(" g= {} ", g(" hello "));
}

In [31]:
main()

 g= h 


()